# Neighborhood Based Recommender Systems
Here we implement a user-based recommender system on the MovieLens 100k Dataset.

In [1]:
# Importing packages
import numpy as np
import pandas as pd
import math

In [2]:
# Reads in the list of items from .\\ml-100k\\u.item

items_pd = pd.read_table(".\\ml-100k\\u.item", header=None, names=['item id', 'title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy'
    , 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], delimiter='|', encoding = "ISO-8859-1")
items_pd.head()
ITEMS = items_pd.shape[0]

# Reads in the list of items from .\\ml-100k\\u.user

users_pd = pd.read_table(".\\ml-100k\\u.user", header=None, names=['user id', 'age', 'gender', 'occupation', 'zip code'], delimiter='|', encoding = "ISO-8859-1")
users_pd.head()
USERS = users_pd.shape[0]

## u1 data set
Here we work with the u1 data set.

In [3]:
# Here we read in the ratings in u1.base

u1_base_pd = pd.read_table(".\\ml-100k\\u1.base", header=None, names=['user id', 'item id', 'rating', 'timestamp'])
u1_base_pd.head()

# rows in u1_base_pd hold data in the form <user id, item id, rating, timestamp>
# each row represents a specific rating

,user id,item id,rating,timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712


In [4]:
# This function converts the ratings read from u1.base into a matrix with users as rows and items as columns

def to_data_pd(input):
    data_array = np.empty((USERS, ITEMS))
    for _, rating in input.iterrows():
        data_array[rating['user id'] - 1][rating['item id'] - 1] = rating['rating']
    output_pd = pd.DataFrame(data_array, index=range(1,USERS+1), columns=range(1,ITEMS+1))
    return output_pd
    

In [5]:
# put u1_base_pd in a usable format

u1_data_pd = to_data_pd(u1_base_pd)
u1_data_pd.head()

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.0,3.0,4.0,3.0,3.0,0.0,4.0,1.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# This function finds the average rating for each user

def find_average(data_pd):
    avgs = []
    for _, user in data_pd.iterrows():
        avgs.append(user.sum() / user.astype(bool).sum())

    return avgs

In [49]:
# find each user's average rating

averages = find_average(u1_data_pd)

# find which items each user rated

rated_items = [{k for k in range(1,ITEMS+1) if u1_data_pd[k][i] > 0} for i in range(1,USERS+1)]

Now that most of the prep work is done, it's time to calculate the matrix of Pearson correlation values.

In [17]:
#This implements the Pearson metric as found in equation (2.2)
def pearson(user1, user2):
    intersection = rated_items[user1 - 1].intersection(rated_items[user2 - 1])
    if len(intersection) <= 1:
        return 0

    bottom1 = math.sqrt( sum([(u1_data_pd.iloc[user1-1][k] - averages[user1-1]) ** 2 for k in intersection]))
    bottom2 = math.sqrt( sum([(u1_data_pd.iloc[user2-1][k] - averages[user2-1]) ** 2 for k in intersection]))

    if bottom1 * bottom2 == 0:
        return 0
        
    top = sum([(u1_data_pd.iloc[user1-1][k] - averages[user1-1]) * (u1_data_pd.iloc[user2-1][k] - averages[user2-1]) for k in intersection])

    return top / (bottom1 * bottom2)

In [19]:
# I don't know if this is the way to go about this.
# It takes a long time to run. However, I think it'set
# necessary because we'll have to calculate these values
# sooner or later anyways.

pearson_matrix = np.empty((USERS,USERS))
for i in range(1, USERS+1):
    print(i)
    # since the matrix is symmetric, we can save time by only calculating half the values
    for j in range(i, USERS+1):
        if i == j:
            pearson_matrix[i - 1][j - 1] = 1
        pm = pearson(i,j)
        pearson_matrix[i - 1][j - 1] = pm
        pearson_matrix[j - 1][i - 1] = pm

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [31]:
# save the results in a csv so we don't have to ever run it again

u1_pearson_df = pd.DataFrame(pearson_matrix, columns=[i for i in range(1,USERS+1)])
u1_pearson_df.index = np.arange(1, USERS + 1)
u1_pearson_df.to_csv(r'.\u1_pearsonmatrix.csv')

In [11]:
# okay let's try to multithread this

# import threading

# THREADCOUNT = 4

# def pearson_matrix_threaded(outputs, i_range):
#     output_np = np.zeros((USERS,USERS))
#     for i in i_range:
#         for j in range(1, USERS):
#             pm = pearson(i,j)
#             print(i, j, pm)
#             output_np[i - 1][j - 1] = pm
#     outputs.append(pd.DataFrame(output_np))

# outputs = []
# threads = [threading.Thread(target=pearson_matrix_threaded, args=(outputs, range(k, USERS, THREADCOUNT),)) for k in range(THREADCOUNT)]
# for thread in threads:
#     thread.start()

# for thread in threads:
#     thread.join()


# df = pd.concat(outputs)
# df.head()

# df.to_csv("pearsonmatrix.csv")




'\nimport threading\n\nTHREADCOUNT = 4\n\ndef pearson_matrix_threaded(outputs, i_range):\n    output_np = np.zeros((USERS,USERS))\n    for i in i_range:\n        for j in range(1, USERS):\n            pm = pearson(i,j)\n            print(i, j, pm)\n            output_np[i - 1][j - 1] = pm\n    outputs.append(pd.DataFrame(output_np))\n\noutputs = []\nthreads = [threading.Thread(target=pearson_matrix_threaded, args=(outputs, range(k, USERS, THREADCOUNT),)) for k in range(THREADCOUNT)]\nfor thread in threads:\n    thread.start()\n\nfor thread in threads:\n    thread.join()\n\n\ndf = pd.concat(outputs)\ndf.head()\n\ndf.to_csv("pearsonmatrix.csv")\n\n'

In [69]:
u1_pearson_df = pd.read_csv(r".\u1_pearsonmatrix.csv", usecols=[i for i in range(1,USERS+1)], header=0)
u1_pearson_df.index = np.arange(1, USERS + 1)
u1_pearson_df.head()

,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
1,1.000000,0.521808,0.525129,1.000000,0.147819,0.298258,0.227549,0.874981,0.907111,-0.044601,...,0.053592,-0.438547,0.486317,-0.181461,0.554811,0.154184,0.250645,0.813325,-0.177845,0.296437
2,0.521808,1.000000,0.140028,-0.514496,0.000000,0.385681,0.775999,0.000000,0.264764,-0.301511,...,0.049314,-0.282424,0.307308,0.644691,0.209520,0.457652,-0.080313,0.032796,0.249705,0.712488
3,0.525129,0.140028,1.000000,0.000000,0.000000,-0.655786,-1.000000,0.435194,0.000000,0.756978,...,0.000000,0.000000,-0.176824,0.356291,-0.244086,-0.424907,-0.108180,0.739940,0.513321,0.000000
4,1.000000,-0.514496,0.000000,1.000000,0.000000,0.000000,-0.497343,0.978363,0.000000,0.000000,...,0.000000,0.000000,-0.210754,-0.256307,0.143209,0.000000,0.839260,0.921244,0.358067,-0.863779
5,0.147819,0.000000,0.000000,0.000000,1.000000,-0.034966,0.249292,0.491733,-0.290883,-0.216527,...,0.323622,-0.622744,-0.224290,0.951357,0.562100,0.913966,0.224538,0.754131,0.249817,0.335129


In [34]:
# Get the set of users that rated each item

rated_users = [{k for k in range(1,USERS+1) if u1_data_pd[i][k] > 0} for i in range(1,ITEMS+1)]

In [56]:
print((rated_users[5]))

{773, 524, 655, 18, 662, 537, 924, 806, 936, 181, 568, 828, 707, 198, 71, 76, 590, 486, 870, 758}


In [87]:
# Of the users who rated item, return the k with highest pearson correlation value with a specified user

def k_closest(user, item, k):
    # pearson_sorted = sorted([u1_pearson_df.iloc[user - 1][i] for i in range(1, USERS+1) if i in rated_users[item - 1]])
    pearson_sorted = sorted((i for i in range(1, USERS+1) if i in rated_users[item - 1]), key=lambda x: u1_pearson_df[str(user)][x])[-k:]
    return pearson_sorted

In [88]:
# Implement r-hat as in equation 2.4

def r_hat(user, item):
    K = k_closest(user, item, 5)
    top = sum(u1_pearson_df[str(user)][user2] * u1_data_pd[item][user2] - averages[user2-1] for user2 in K)
    bot = sum(abs(u1_pearson_df[str(user)][user2]) for user2 in K)

    return averages[user-1] + top / bot

r_hat(4,1)

4.205656838316722